In [ ]:
""" IN THE NAME OF  ALLAH , THE MOST GRACIOUS, THE MOST MERCIFUL. """

In [1]:
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.models import resnet18
from torch import Tensor
from torch.utils.data import DataLoader, random_split

modules_path = 'C:/Users/PMLS/FYP/Modules/'
dataset_path = 'C:/Users/PMLS/FYP/Dataset/STMM'
save_path = "C:/Users/PMLS/FYP/Models"

sys.path.insert(1, modules_path)
from video_dataset import VideoFrameDataset, ImglistToTensor

if False and torch.xpu.is_available():
    device = torch.device("xpu")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

C:\Users\PMLS\anaconda3\Lib\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\PMLS\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


cpu


In [2]:
# Load dataset
#img_w = 640
#img_h = 480
#frames_per_video = 7
batch_size = 10

dataset = VideoFrameDataset(
    root_path= f"{dataset_path}",
    annotationfile_path=f"{dataset_path}/annotations.txt",
    num_segments=8,
    frames_per_segment=1,
    imagefile_template='{:01d}.jpg',
    transform=ImglistToTensor(),
    test_mode=False
)
class_map = {0:"down", 1:"left", 2:"right", 3:"up"}

train_dataset, valid_dataset, test_dataset = random_split(dataset, [0.7,0.2,0.1])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=True)

In [11]:
# Define model
class CNN_LSTM(nn.Module):
    def __init__(self, num_classes=4):
        super(CNN_LSTM, self).__init__()
        self.resnet = resnet18(pretrained=True)
        #self.resnet.fc = nn.Sequential(nn.Linear(self.resnet.fc.in_features, 300))
        self.lstm = nn.LSTM(input_size=1000, hidden_size=256, num_layers=3)
        self.linear1 = nn.Linear(256, 128)
        self.linear2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        out1 = self.resnet(x)
        out2 = self.lstm(out1)[0]
        out3 = self.linear1(out2)
        out4 = self.linear2(out3)
        return out4 

model = CNN_LSTM()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_function = torch.nn.CrossEntropyLoss()

C:\Users\PMLS\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\PMLS\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
print(model)

In [25]:
# Define transforms
import cv2
import pywt
def wavelet_transform(batch):
        batch2=[]
        for frame in batch:
            A, B = pywt.dwt(frame, 'db1')
            batch2.append(B)
        return np.array(batch2)

def grey_transform(batch):
    batch = batch.transpose(0,1,3,4,2)
    favg = np.average(batch[0], axis=0)
    grey_avg = cv2.cvtColor(favg, cv2.COLOR_BGR2GRAY)
    batch2 = []
    for frame in batch[0]:
        frame2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame2 = frame2 - grey_avg
        threshhold = np.max(frame2) * 0.05
        _, frame2 = cv2.threshold(frame2, threshhold, 255, cv2.THRESH_BINARY)
        batch2.append(frame2)
    return np.array([batch2])
    
def local_transform(batch):
    batch = np.array(batch.cpu())
    new_batch = batch.transpose(1,0,2,3,4)
    return torch.tensor(np.array(new_batch))

In [27]:
# Define training loop
def train(model, data):
    model.train()
    for i, (x, y) in enumerate(data):
        #print("\r"+str(i), end="")
        x = local_transform(x)
        x = x.to(device)
        y = y.to(device)

        for x_j in x:
            y_hat = model(x_j)
            loss = loss_function(y_hat, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
    return loss.item()

def test(model, data, label=""):
    correct = 0
    model.eval()
    with torch.no_grad():
        for i, (x,y) in enumerate(data):
            #x = local_transform(x)
            x = x.flatten(start_dim=1).to(device)
            y = y.to(device)
            
            y_hat = model(x)
            _, y_hat = torch.max(y_hat,1)
            correct += (y_hat==y).sum()
    print(label+"accuracy:", round(float(correct/(i+1)/batch_size), 3))

In [ ]:
%%time

# Train
N=1
for epoch in range(N):
    loss = train(model, train_loader)
    print("  Epoch:",epoch, " Loss:", round(loss,3))